<a href="https://colab.research.google.com/github/svg0016/PythonProject/blob/main/AmazonStock2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This program uses 

#Import Libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


#Get the stock quote
df = web.DataReader('AMZN', data_source='yahoo', start='2018-01-01', end= '2020-12-16')

#Show the data
df



In [ ]:
#Get the number of rows and colums in the data set
df.shape

In [ ]:
#Visualize the closing price history
plt.figure(figsize=(16,8))
plt.title('Closing Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize= 18)
plt.ylabel('ClosingPrice ($)USD', fontsize= 18)
plt.show()

In [ ]:
#Create a new dataframe with only the 'Close column
data = df.filter(['Close'])
#Convert the dataframe to a numpy array
dataset = data.values
#Get the number of rows to train the model on
training_data_len = math.ceil( len(dataset)*.80 )

training_data_len

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [684]:
#Create the traiing data set
#Creat the scaled training data set
train_data = scaled_data[0:training_data_len, :]
#Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(50, len(train_data)):
    x_train.append(train_data[i-50:i, 0])
    y_train.append(train_data[i,0])

    if i<=51:
        print(x_train)
        print(y_train)
        print()


[array([0.        , 0.00648467, 0.00878569, 0.01713171, 0.02470073,
       0.02718957, 0.02788543, 0.03742681, 0.0496021 , 0.04945697,
       0.04524769, 0.04453046, 0.04506837, 0.05904102, 0.07408089,
       0.07193354, 0.08065946, 0.09094792, 0.09762045, 0.10621828,
       0.11179796, 0.08580369, 0.10285854, 0.08580369, 0.10836135,
       0.09723623, 0.06894093, 0.06428765, 0.08419425, 0.09626714,
       0.11186628, 0.11643842, 0.11085874, 0.11925171, 0.12547175,
       0.12650483, 0.13276327, 0.14213382, 0.13787759, 0.13807822,
       0.12996702, 0.13287   , 0.1428425 , 0.14883199, 0.15197401,
       0.15490257, 0.16644184, 0.17476649, 0.1704078 , 0.17161165])]
[0.1679060920927259]

[array([0.        , 0.00648467, 0.00878569, 0.01713171, 0.02470073,
       0.02718957, 0.02788543, 0.03742681, 0.0496021 , 0.04945697,
       0.04524769, 0.04453046, 0.04506837, 0.05904102, 0.07408089,
       0.07193354, 0.08065946, 0.09094792, 0.09762045, 0.10621828,
       0.11179796, 0.08580369, 0.102

In [685]:
#Convert the x_train and y_train to numpy arrays
x_train,y_train = np.array(x_train), np.array(y_train)

In [686]:
#Reshape the data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape
#x_test.shape[0] = number of rows
#x_test.shape[1] = number of columns/timestamps
#1 = number of features, close price

(547, 50, 1)

In [687]:
#Build the LSTM(Long Term Short Memory) model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

In [688]:
#Compile the model
model.compile(optimizer= 'adam', loss='mean_squared_error')


In [689]:
#Train the Model
model.fit(x_train, y_train, batch_size=1, epochs=1)

547/547 [==============================] - 15s 23ms/step - loss: 0.0036


In [695]:
#Create the testing data set
#Create a new array conaining scaled values from index 748 997
test_data = scaled_data[training_data_len - 50: , :]
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]

for i in range(50, len(test_data)):
  x_test.append(test_data[i-50:i, 0])



In [697]:
#Convert the data to a numpy array
x_test = np.array(x_test)


In [698]:
#Reshape the Data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
#x_test.shape[0] = number of rows
#x_test.shape[1] = number of columns/timestamps
#1 = number of features, close price


In [699]:
#Get the models predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)#Unscaling the predictions


In [700]:
#Get the root mean squared error(RMSE)
rmse = np.sqrt( np.mean(predictions - y_test )**2)
rmse

67.12186385801175

In [ ]:
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date')
plt.ylabel('Closing Price $USD')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train','Actual','Predictions'],loc= 'lower right' )
plt.show()

In [599]:
#Show the valid and predicted prices

valid


,Close,Predictions
Date,,
2020-03-06,1901.089966,1989.543213
2020-03-09,1800.609985,1979.281616
2020-03-10,1891.819946,1961.139526
2020-03-11,1820.859985,1948.027466
2020-03-12,1676.609985,1932.581299
...,...,...
2020-12-10,3101.489990,3102.270264
2020-12-11,3116.419922,3094.195801
2020-12-14,3156.969971,3086.898193


In [702]:
#Get the quote
amzn_quote = web.DataReader('AMZN', data_source='yahoo', start='2014-01-01', end='2020-12-16')
#Create a new dataframe
new_df = amzn_quote.filter(['Close'])
#Get the last 60 day closing price values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append the past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

[[3246.612]]


In [703]:
#Get Amazon Current value
amzn_quote2 = web.DataReader('AMZN', data_source='yahoo', start='2020-12-17', end='2020-12-17')
print(amzn_quote2['Close'])

Date
2020-12-17    3236.080078
Name: Close, dtype: float64
